In [202]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [203]:
Transactions=pd.read_excel('2022 Case challenge_ DA update.xlsx',sheet_name='Data. Transactions')

In [204]:
Transactions=Transactions[['User_id','Order_id','DATE','GMV','Service Group','MerchantID']]


In [205]:
Transactions['User_id']=Transactions['User_id'].astype(str)
Transactions['Order_id']=Transactions['Order_id'].astype(str)
Transactions['MerchantID']=Transactions['MerchantID'].astype(str)

In [206]:
Transactions.head()

,User_id,Order_id,DATE,GMV,Service Group,MerchantID
0,61386143,8733622706,2021-01-01,100000,marketplace,37
1,48453125,8726857991,2021-01-01,5000,supermarket,9
2,49921027,8737326894,2021-01-01,106600,supermarket,9
3,46022523,8732579078,2021-01-01,270000,supermarket,9
4,44014594,8725567343,2021-01-01,68000,cvs,8


In [207]:
Loyalty_Points=pd.read_excel('2022 Case challenge_ DA update.xlsx',sheet_name='Data. Loyalty Points')

In [208]:
Loyalty_Points

,Service Group,Point Mechanism,Maximum Point Per Trans
0,supermarket,1 points/1000VND GMV,500 points
1,data,10 points/1000VND GMV,1000 points
2,cvs,1 points/1000VND GMV,300 points
3,marketplace,1 points/1000VND GMV,500 points
4,Coffee chains and Milk tea,1 points/1000VND GMV,500 points
5,Offline Beverage,1 points/1000VND GMV,300 points


In [209]:
Loyalty_Benefits=pd.read_excel('2022 Case challenge_ DA update.xlsx',sheet_name='Data. Loyalty benefits')

In [210]:
Loyalty_Benefits

,Class ID,Group,%cashback
0,2,cvs,5
1,2,Offline Beverage,5
2,2,data,5
3,3,cvs,7
4,3,Offline Beverage,7
5,3,data,7
6,3,marketplace,2
7,3,supermarket,2
8,4,cvs,12
9,4,Offline Beverage,12


# Part 1

1

In [211]:
new_Loyalty_Points=Loyalty_Points[['Service Group','Maximum Point Per Trans']]
new_Loyalty_Points['Maximum Point Per Trans']=new_Loyalty_Points['Maximum Point Per Trans'].apply(lambda x: x.split(' ')[0])
Transactions=Transactions.merge(new_Loyalty_Points,on='Service Group',how='left')

C:\Users\ACER\AppData\Local\Temp\ipykernel_10016\1909542948.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_Loyalty_Points['Maximum Point Per Trans']=new_Loyalty_Points['Maximum Point Per Trans'].apply(lambda x: x.split(' ')[0])


In [212]:
Transactions['Loyalty Points 1'] = Transactions.apply(lambda row: int(row['GMV'] / 1000 * 10) if row['Service Group'] == 'data' else int(row['GMV'] / 1000 * 1), axis=1)


In [213]:
Transactions.head()

,User_id,Order_id,DATE,GMV,Service Group,MerchantID,Maximum Point Per Trans,Loyalty Points 1
0,61386143,8733622706,2021-01-01,100000,marketplace,37,500,100
1,48453125,8726857991,2021-01-01,5000,supermarket,9,500,5
2,49921027,8737326894,2021-01-01,106600,supermarket,9,500,106
3,46022523,8732579078,2021-01-01,270000,supermarket,9,500,270
4,44014594,8725567343,2021-01-01,68000,cvs,8,300,68


In [214]:
Transactions['Maximum Point Per Trans']=Transactions['Maximum Point Per Trans'].astype(int)

In [215]:
Transactions['Loyalty Points']=Transactions.apply(lambda row : row['Loyalty Points 1'] if row['Loyalty Points 1'] < row['Maximum Point Per Trans'] else row['Maximum Point Per Trans'],axis=1)

In [216]:
Transactions.drop(['Maximum Point Per Trans','Loyalty Points 1'],axis=1,inplace=True)

In [217]:
Transactions.head()

,User_id,Order_id,DATE,GMV,Service Group,MerchantID,Loyalty Points
0,61386143,8733622706,2021-01-01,100000,marketplace,37,100
1,48453125,8726857991,2021-01-01,5000,supermarket,9,5
2,49921027,8737326894,2021-01-01,106600,supermarket,9,106
3,46022523,8732579078,2021-01-01,270000,supermarket,9,270
4,44014594,8725567343,2021-01-01,68000,cvs,8,68


In [218]:
current_date = datetime.datetime(2022, 3, 31)
days_to_reset = 30
reset_date = current_date - datetime.timedelta(days=days_to_reset)
Transactions.loc[Transactions['DATE'] < reset_date, 'Loyalty Points'] = 0


In [219]:
Loyalty_Ranking=Transactions.groupby('Order_id')['Loyalty Points'].sum().reset_index()

In [220]:
Loyalty_Ranking.head()

,Order_id,Loyalty Points
0,10003480552,0
1,10004582770,0
2,10004698228,0
3,10005708329,0
4,10005986271,0


In [221]:
Loyalty_Ranking.drop(Loyalty_Ranking[Loyalty_Ranking['Loyalty Points'] == 0].index, inplace=True)


In [222]:
Loyalty_Ranking['Rank_name']=Loyalty_Ranking['Loyalty Points'].apply(lambda x: 'DIAMOND' if x>=5000 
                                                                     else('GOLD' if x<5000 and x>=2000 
                                                                          else('SILVER' if x<2000 and x >=1000 else 'STANDARD')))

In [223]:
Loyalty_Ranking.rename(columns={'Loyalty Points':'Calculated_points'},inplace=True)

In [224]:
Loyalty_Ranking.head()

,Order_id,Calculated_points,Rank_name
41629,21483226154,29,STANDARD
41806,21500227291,294,STANDARD
41807,21500415960,140,STANDARD
41808,21501509605,300,STANDARD
41809,21501920478,1,STANDARD


In [225]:
Loyalty_Ranking['Rank_name'].value_counts()

STANDARD    6434
SILVER       275
Name: Rank_name, dtype: int64